# <center> **k-NN Vizinhos** <center>

Guerreiros: **Tsuru** (Júlia Guedes Almeida dos Santos), **Pólux** (Raquel de Godoy Vianna) e **Tiles** (Thalles José de Souza Cansi).

> Pimenta nos olhos dos outros é refresco.

## 🌇 **Desenvolvimento - Modelo k-NN Vizinhos**

<div style="text-align: justify;">
Os notebooks de desenvolvimento visam descrever e implementar os diferente modelos de aprendizado de máquina a serem testados no banco de dados definido a priori. Neste projeto, serão implementados os seguintes modelos: Regressão Linear Padrão, Descida do Gradiente Estocástico (SGD), Árvore de Decisão, Floresta Aleatória e k-NN Vizinhos. Vale ressaltar que a otimização deles foi realizada anteriormente no notebook denominado "Otimização de hiperparâmetros e Redução de dimensionalidade" [1] e encontra-se disponível para ser acessado no Github do trabalho. [2]
<div>

### 🖼️ **Introdução**

O modelo preditivo k-NN vizinhos pauta-se na teoria de que dados similares encontram-se em regiões próximas no espaço dos atributos, resumidamente, ele se baseia na distância entre o objeto de entrada e os que foram previamente utilizados no treinamento do modelo (vizinhos) em certo plano. Nesse sentido, o valor predito consiste na média dos targets dos $k$ vizinhos mais próximos do objeto de entrada, ou seja, os $k$ vizinhos que apresentaram menor distância em relação a esse.

### 📚 **Importação de bibliotecas**

Em primeiro lugar, as bibliotecas necessárias serão importadas abaixo.

In [53]:
import os
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
import numpy as np

### ⚛️ **Importação dos Dataframes**

O dataframe de supercondutividade será definido e, sequencialmente, exibido. Além disso, os dataframes da otimização do modelo k-NN vizinhos, assim como o dataframe que contém as métricas dos outros modelos que já foram implementados e os dataframes de treino e teste serão importados a seguir.

In [36]:
df = pd.read_csv(f"{os.getcwd()}\\dataset_condutividade.csv", sep = ';')
display(df)

,number_of_elements,mean_atomic_mass,wtd_mean_atomic_mass,gmean_atomic_mass,wtd_gmean_atomic_mass,entropy_atomic_mass,wtd_entropy_atomic_mass,range_atomic_mass,wtd_range_atomic_mass,std_atomic_mass,...,wtd_mean_Valence,gmean_Valence,wtd_gmean_Valence,entropy_Valence,wtd_entropy_Valence,range_Valence,wtd_range_Valence,std_Valence,wtd_std_Valence,critical_temp
0,4,88.944468,57.862692,66.361592,36.116612,1.181795,1.062396,122.90607,31.794921,51.968828,...,2.257143,2.213364,2.219783,1.368922,1.066221,1,1.085714,0.433013,0.437059,29.00
1,5,92.729214,58.518416,73.132787,36.396602,1.449309,1.057755,122.90607,36.161939,47.094633,...,2.257143,1.888175,2.210679,1.557113,1.047221,2,1.128571,0.632456,0.468606,26.00
2,4,88.944468,57.885242,66.361592,36.122509,1.181795,0.975980,122.90607,35.741099,51.968828,...,2.271429,2.213364,2.232679,1.368922,1.029175,1,1.114286,0.433013,0.444697,19.00
3,4,88.944468,57.873967,66.361592,36.119560,1.181795,1.022291,122.90607,33.768010,51.968828,...,2.264286,2.213364,2.226222,1.368922,1.048834,1,1.100000,0.433013,0.440952,22.00
4,4,88.944468,57.840143,66.361592,36.110716,1.181795,1.129224,122.90607,27.848743,51.968828,...,2.242857,2.213364,2.206963,1.368922,1.096052,1,1.057143,0.433013,0.428809,23.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21258,4,106.957877,53.095769,82.515384,43.135565,1.177145,1.254119,146.88130,15.504479,65.764081,...,3.555556,3.223710,3.519911,1.377820,0.913658,1,2.168889,0.433013,0.496904,2.44
21259,5,92.266740,49.021367,64.812662,32.867748,1.323287,1.571630,188.38390,7.353333,69.232655,...,2.047619,2.168944,2.038991,1.594167,1.337246,1,0.904762,0.400000,0.212959,122.10
21260,2,99.663190,95.609104,99.433882,95.464320,0.690847,0.530198,13.51362,53.041104,6.756810,...,4.800000,4.472136,4.781762,0.686962,0.450561,1,3.200000,0.500000,0.400000,1.98
21261,2,99.663190,97.095602,99.433882,96.901083,0.690847,0.640883,13.51362,31.115202,6.756810,...,4.690000,4.472136,4.665819,0.686962,0.577601,1,2.210000,0.500000,0.462493,1.84


In [37]:
df_otimizacao_knn = pd.read_csv(f"{os.getcwd()}\\df_otimizacao_knn.csv")

In [38]:
df_RMSE = pd.read_csv(f"{os.getcwd()}\\df_RMSE.csv")

In [39]:
df_teste_features = pd.read_csv(f"{os.getcwd()}\\df_teste_features.csv")

In [40]:
df_teste_target = pd.read_csv(f"{os.getcwd()}\\df_teste_target.csv")

In [41]:
df_treino_target = pd.read_csv(f"{os.getcwd()}\\df_treino_target.csv")

In [42]:
df_treino_features = pd.read_csv(f"{os.getcwd()}\\df_treino_features.csv")

### 🎯**Dados de treino e de teste**

<div style="text-align: justify">
Tendo em vista a divisão realizada no notebook introdutório, serão estabelecidos os arrays dos dados de treino e os dos de teste, os quais serão cruciais para implementar o modelo k-NN vizinhos.
</div>

In [43]:
X_treino = df_treino_features.values
y_treino = df_treino_target.values.ravel()

X_teste = df_teste_features.values
y_teste = df_teste_target.values.ravel()

### 💱 **Normalização dos dados**

A normalização de dados, de maneira breve, consiste na transformação dos dados em uma escala em comum. No trabalho em questão, será utilizada a normalização padrão, que altera o desvio padrão dos dados ($\sigma$) para 1 a média deles ($\mu$) para 0.
A equação dessa normalização de dados encontra-se exibida abaixo:

$$z_i = \frac{x_i - \mu}{\sigma}$$

<center> [Fórmula 1: Normalização padrão]

Nela, $x_i$ consiste no exemplo $i$ dentre os dados $x$ e $z_i$ representa o valor normalizado de $x_i$.

Como o modelo k-NN é pautado em uma métrica de distância entre pontos, é essencial que os dados sejam normalizados antes da implementação, para que as distâncias entre os atributos sejam comparáveis. (Discorre-se melhor acerca de tal conceito no notebook "Quest 1 - Normalizadores" - contido na pasta de quests do github - onde esse tópico é mais explorado). [5]

Nesse sentido, mais adiante, será implementado por intermédio da biblioteca `scikit-learn`, a normalização padrão sobre os dados, utilizando `StandardScaler`.

### 🏘️ **k-NN Vizinhos**

Agora, com o intuito de implementar tal modelo visando a melhor métrica possível, o número $k$ de vizinhos, bem como o peso serão definidos levando em consideração o notebook de otimização de parâmetros [1]. Assim, abaixo é exibido o dataframe definido no notebook em questão para a otimização do modelo k-NN vizinhos.

In [44]:
display(df_otimizacao_knn)

,Unnamed: 0,n_neighbors,weights
0,0,4,distance


Os resultados obtidos exprimem que o número de vizinhos (n_neighbors) deve ser igual a 4, enquanto o peso (weights) a ser considerado deve ser a distância (distance). Além disso, haja vista que o conjunto de dados escolhido apresenta diversos atributos, a métrica de distância utilizada será a distância Manhattan (p=1) ao invés da distância Euclidiana, visto que a última não é a ideal para levar em consideração pontos com mais de três coordenadas, situação evidenciada com o dataframe selecionado para tal projeto. Tal constatação encontra-se mais detalhadamente explicada no notebook denominado "Modelos 1" contido na seção das quests 2 no github do projeto. [3,6]

Com base no que foi estabelecido a priori, o modelo apresentado no presente notebook será implementado abaixo, logo após a normalização dos dados por intermédio de um pipeline, bem como da biblioteca `scikit-learn`, das funções `KNeighborsRegressor`, `StandardScaler` e `make_pipeline`.

In [47]:
modelo_knn_pipeline = make_pipeline(
    StandardScaler(),
    KNeighborsRegressor(n_neighbors=4, p=1, weights='distance')
)
modelo_knn_pipeline.fit(X_treino, y_treino)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('kneighborsregressor',
                 KNeighborsRegressor(n_neighbors=4, p=1, weights='distance'))])

Após a implementação do modelo, resta estimar a performance do modelo por meio da validação cruzada.

In [54]:
scores_rl= cross_val_score(modelo_knn_pipeline, X_teste, y_teste, cv=10, scoring="neg_root_mean_squared_error")
RMSE_baseline = np.mean(scores_rl)
print(f"A média das métricas de RMSE do modelo k-NN vizinhos foi de {-RMSE_baseline} K.")

A média das métricas de RMSE do modelo k-NN vizinhos foi de 12.468320433763953 K.


Por fim, o valor obtido acima será armazenado no dataframe que contém as estimativas das métricas de todos os modelos que já foram testados (df_RMSE).

In [55]:
df_RMSE["knn_vizinhos"] = -RMSE_baseline

In [56]:
display(df_RMSE)

,Unnamed: 0,Baseline,knn_vizinhos
0,0,33.887487,12.46832


<div style="text-align: justify">

Com a explicação e implementação do modelo k-NN vizinhos, é evidente que tal predição apresentou um RMSE melhor que o do baseline! Agora, só falta a comparação desses resultado com as métricas que virão dos próximos modelos analisados... A Aliança da Supernova instrui que seja explorado por vossas senhorias, caros leitores atentos, outros notebooks de desenvolvimento, ou, então, o de conclusão. Esperamo-vos lá!

</div>

``Que a sede pela exploração da magia da predição esteja sempre com vocês! ⚔️``

## 📚 **Referências!**

[1] Santos, J.G.A; Vianna, R.G.; Cansi, T.J.S. (2024). Jupyter Notebook *Otimização de hiperparâmetros e Redução de dimensionalidade*

[2] Github do projeto: https://github.com/JuliaGuedesASantos/Machine-Learning---Cavaleiros-da-Supernova

[3] Daniel Roberto Cassar. (2024). Jupyter Notebook *ATP-203 2.1 - Aprendizado de máquina, k-NN e métricas*. [Material não publicado].

[4] Documentação scikit-learn KNeighborsRegressor: https://scikit-learn.org/1.5/modules/generated/sklearn.neighborsKNeighborsRegressor.html

[5] Santos, J.G.A; Vianna, R.G.; Cansi, T.J.S. (2024). Jupyter Notebook *Quest 1 - Normalizadores*

[6] Santos, J.G.A; Vianna, R.G.; Cansi, T.J.S. (2024). Jupyter Notebook *Modelos 1*

[7] Daniel Roberto Cassar. (2024). Jupyter Notebook *ATP-203 2.1 - Tratamento de dados*. [Material não publicado].